In [1]:
# Clear memory
%reset -f

In [2]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import ipywidgets
ipywidgets.Widget.close_all()

import SimpleITK
import datetime
import gc # Garbage collected
import numpy as np
import sys, os
import time
import tifffile
import matplotlib.pyplot as plt
import scipy
from tkinter import Tk
from tkinter.filedialog import askopenfilename, askopenfilenames, askdirectory
import torch
import skimage
from skimage import measure
from skimage.morphology import skeletonize
import pickle
import networkx as nx
import sknw # https://github.com/Image-Py/sknw

import multiprocessing

this_path = os.getcwd()
print(this_path)


import h5py
# import File
from scripts import *
import wrapper_ASTRA

/global/u2/e/eboigne/tomoTools_als832/tomoTools


In [3]:
path_save = '/global/homes/e/eboigne/cfs_als/2022_wood/'

# case = 'run21_oak_lowHeat'
# case = 'run23_walnut_lowHeat'
case = 'run24_birch_lowHeat'
# case = 'run26_birch_highHeat'

voxel_size = 3.24*2 # [microns]

list_cases_folder_name = sorted([e for e in os.listdir(path_save+case) if not 'probe' in e and not '.tif' in e and not '.pickle' in e])
run = case[3:5]+'_Sample'
list_cases_h5 = sorted([e for e in os.listdir(path_save) if '.h5' in e and run in e])

print(list_cases_folder_name)

['01', '02', '03', '04', '05', '06', '07', '08a', '08b', '09a', '09b', '10a', '10b', '11a', '11b', '12a', '12b', '13a', '13b', '14a', '14b', '15a', '15b', '16a', '16b', '17a', '17b', '18a', '18b', '19a', '19b', '20a', '20b', '21a', '21b', '22', '23', '24']


In [4]:
ind_slice = 250

slices = []
for ind_case, this_case in enumerate(list_cases_folder_name[:]):
    slices.append(File(path_save+case+'/'+this_case+'/b_movingRegisteredToStatic/').read(ind_slice))
    print(ind_case, this_case)

slices = np.array(slices)
print(slices.shape)
File(path_save+case+'/probe_slice_'+str(ind_slice).zfill(4)).saveTiffStack(slices)

0 01
1 02
2 03
3 04
4 05
5 06
6 07
7 08a
8 08b
9 09a
10 09b
11 10a
12 10b
13 11a
14 11b
15 12a
16 12b
17 13a
18 13b
19 14a
20 14b
21 15a
22 15b
23 16a
24 16b
25 17a
26 17b
27 18a
28 18b
29 19a
30 19b
31 20a
32 20b
33 21a
34 21b
35 22
36 23
37 24
(38, 1280, 1280)


In [12]:
c = []
for this_case in enumerate(list_cases_h5):
    c.append(this_case[1][35:37])

t = []

list_scans = []

for this_case in enumerate(c):
    if str(this_case[1]) in list_cases_folder_name:
        cc = list_cases_h5[this_case[0]][9:15]
        h = cc[0:2]
        m = cc[2:4]
        s = cc[4:]
        ts = float(int(h)*3600+int(m)*60+int(s))
        t.append(ts) #c
        print(len(t), this_case, 'c', ts)
    else:
        cc = list_cases_h5[this_case[0]][9:15]
        h = cc[0:2]
        m = cc[2:4]
        s = cc[4:]
        ts0 = float(int(h)*3600+int(m)*60+int(s))
        t.append(ts0) #a
        ts1 = 26.26 + ts0
        t.append(ts1) #b
        print(len(t), this_case, 'a/b', ts0 ,ts1)

scan_times = np.array(t) - t[0]

assert(len(list_cases_folder_name) ==len(scan_times)), 'Wrong sizes'

for ind_scan,scan_time in enumerate(scan_times):
    this_scan = Data()
    this_scan.scan_time = scan_time
    this_scan.folder_name = list_cases_folder_name[ind_scan]
    this_scan.full_path = path_save+case+'/'+this_scan.folder_name
    this_scan.ind = ind_scan
    list_scans.append(this_scan)

1 (0, '01') c 62666.0
2 (1, '02') c 62972.0
3 (2, '03') c 63291.0
4 (3, '04') c 63812.0
5 (4, '05') c 64069.0
6 (5, '06') c 64249.0
7 (6, '07') c 64434.0
9 (7, '08') a/b 64616.0 64642.26
11 (8, '09') a/b 64813.0 64839.26
13 (9, '10') a/b 65005.0 65031.26
15 (10, '11') a/b 65187.0 65213.26
17 (11, '12') a/b 65369.0 65395.26
19 (12, '13') a/b 65553.0 65579.26
21 (13, '14') a/b 65736.0 65762.26
23 (14, '15') a/b 65932.0 65958.26
25 (15, '16') a/b 66121.0 66147.26
27 (16, '17') a/b 66304.0 66330.26
29 (17, '18') a/b 66488.0 66514.26
31 (18, '19') a/b 66673.0 66699.26
33 (19, '20') a/b 66864.0 66890.26
35 (20, '21') a/b 67049.0 67075.26
36 (21, '22') c 67247.0
37 (22, '23') c 67552.0
38 (23, '24') c 67860.0


In [13]:
def compute_mask_solid(this_scan, data, rot_coronal_Fiji, rot_sagittal_Fiji, rot_z_Fiji, force_redo = False):

    if force_redo or not 'b_movingRegisteredToStatic_maskSolid' in os.listdir(this_scan.full_path):

        # Rotating data
        tic = time.time()
        data= rotate_data(data, rot_coronal_Fiji, rot_sagittal_Fiji, rot_z_Fiji)
        print('\tRotating data took: '+str(time.time()-tic))

        File(path_save+case+'/'+this_scan.folder_name+'/b_movingRegisteredToStatic_rotated/').saveTiff(data[:,data.shape[1]//2,:], ind=0)
        File(path_save+case+'/'+this_scan.folder_name+'/b_movingRegisteredToStatic_rotated/').saveTiff(data[:,:,data.shape[2]//2], ind=1)

        threshold = 0.05 # Linear attenuation threshold [/cm], gas to wood
        threshold_needle = 4.0 # Linear attenuation threshold [/cm]
        mask_growth_needle = 10 # Grow the mask obtained using threshold_needle with this amount of pixels
        mask_enclosing_circle = tifffile.imread(path_save+case+'/maskEnclosingCircle.tif') > 0
        bin_factor_dilatation = 1 # Binning before smoothing, as an option to speed things up for large kernels
        filter_half_width = 3 # After binning (equal to sigma for gaussian filter)

        kernel = custom_3d_gaussian_filter(filter_half_width)

        if bin_factor_dilatation > 1:
            data_smoothed = fast_pytorch_bin_3d(data,bin_factor_dilatation, chunk_size = 71)
        else:
            data_smoothed = data

        print('\t3D smoothing for thresholding of mask_solid and mask_needle:')
        data_smoothed = apply_3d_image_processing_on_subvolumes(data_smoothed, fast_pytorch_convolution, kernel_array = kernel, chunk_size_max = (500, 500, 500), overlap = 3*filter_half_width)

        if bin_factor_dilatation > 1:
            data_smoothed = skimage.transform.rescale(data_smoothed, bin_factor_dilatation, multichannel=False)

        mask_needle = data_smoothed > threshold_needle
        mask_needle = apply_3d_image_processing_on_subvolumes(mask_needle, fast_pytorch_mask_dilation, chunk_size_max = (500, 500, 500), overlap = mask_growth_needle, radius = mask_growth_needle)

        mask_solid = data_smoothed > threshold
        mask_solid[mask_needle] = False

        for ind, slice in enumerate(mask_solid):
            slice[~mask_enclosing_circle] = False
            mask_solid[ind] = slice

        print('\tSaving mask_solid and mask_needle:')
        File(this_scan.full_path+'/b_movingRegisteredToStatic_maskNeedle').saveTiffStack(mask_needle, type = 'bool')
        File(this_scan.full_path+'/b_movingRegisteredToStatic_maskSolid').saveTiffStack(mask_solid, type = 'bool')
    else:
        print('\tRe-loading mask solid and mask needle')
        mask_solid = File(this_scan.full_path+'/b_movingRegisteredToStatic_maskSolid').readAll()
        mask_needle = File(this_scan.full_path+'/b_movingRegisteredToStatic_maskNeedle').readAll()

    return(mask_solid.astype('bool'), mask_needle.astype('bool'))

def compute_height(properties):
    for prop in properties:
        bbox = prop.bbox
        prop.height = bbox[3]-bbox[0] # Along z, in pixels
    return(properties)

def compute_equivalent_cylinder_diameters(properties):
    for prop in properties:
        equivalent_cylinder_cross_section = prop.area / prop.height
        prop.equivalent_cylinder_diameter = np.sqrt(4*equivalent_cylinder_cross_section/np.pi) * voxel_size
    return(properties)

def get_path_length(graph, path):
    this_sum=0
    for i in range(len(path)-1):
        this_sum += graph.get_edge_data(path[i], path[i+1])['weight']
    return(this_sum)

def longest_simple_path(graph, source, target):
    longest_path = None
    longest_path_length = 0
    for path in nx.all_simple_paths(graph, source=source, target=target):
        path_length = get_path_length(graph,path)
        if path_length > longest_path_length:
            longest_path_length = path_length
            longest_path = path
    return longest_path, longest_path_length

def find_longest_subgraph_from_ends(graph, bbox):

    ind_end_nodes = [x for x in graph.nodes if graph.degree(x) == 1]
    ind_end_bottom = [x for x in ind_end_nodes if graph.nodes[x]['o'][0] - bbox[0] <=  bbox[3] - graph.nodes[x]['o'][0]]
    ind_end_top = [x for x in ind_end_nodes if not x in ind_end_bottom]

    longest_path_subgraph = None
    longest_path_length = 0
    longest_path = []
    for ind_bot in ind_end_bottom:
        for ind_top in ind_end_top:
            this_longest_path, this_longest_path_length = longest_simple_path(graph, source=ind_bot, target=ind_top)
            this_longest_path_subgraph = graph.subgraph(this_longest_path)

            if this_longest_path_length > longest_path_length:
                longest_path_length = this_longest_path_length
                longest_path = this_longest_path
                longest_path_subgraph = this_longest_path_subgraph

    return(longest_path_subgraph, longest_path, longest_path_length)

def compute_tortuosity(skeleton_properties):

    for ind_pore in range(len(skeleton_properties)):
        bbox = skeleton_properties[ind_pore].bbox
        graph = sknw.build_sknw(skeleton_properties[ind_pore].image, multi=False, iso=False, ring=False, full=True)
        longest_path_subgraph, longest_path, longest_path_length = find_longest_subgraph_from_ends(graph, bbox)

        if longest_path_length == 0:
            longest_path_length = np.nan

        skeleton_properties[ind_pore].length_skeleton = longest_path_length
        skeleton_properties[ind_pore].longest_path = longest_path
        # skeleton_properties[ind_pore].longest_path_subgraph = longest_path_subgraph
        skeleton_properties[ind_pore].tortuosity = longest_path_length / skeleton_properties[ind_pore].height

        # print(ind_pore, longest_path_length, skeleton_properties[ind_pore].tortuosity)
    return(skeleton_properties)

def compute_skeleton(this_scan, mask_pores, force_redo = False):

    if force_redo or not 'b_movingRegisteredToStatic_maskPores_skeleton' in os.listdir(this_scan.full_path):
        skeleton = skeletonize(mask_pores)
        File(this_scan.full_path+'/b_movingRegisteredToStatic_maskPores_skeleton').saveTiffStack(skeleton, type = 'bool')
    else:
        print('\tRe-loading skeleton')
        skeleton = File(this_scan.full_path+'/b_movingRegisteredToStatic_maskPores_skeleton').readAll()
    return(skeleton)

def rotate_data(data, rot_coronal_Fiji, rot_sagittal_Fiji, rot_z_Fiji = 0.0):
    pad = 1
    transform_rotation = sitk.Euler3DTransform()
    center_xyz = (data.shape[0] / 2.0+pad, data.shape[1] / 2.0+pad, data.shape[2] / 2.0+pad)
    center_angle = 0.0
    offsets_xyz = (0.0, 0.0, 0.0)

    transform_rotation.SetFixedParameters((center_xyz[0], center_xyz[1], center_xyz[2], center_angle))
    eulerAngle_xyz_deg = (-rot_sagittal_Fiji, rot_coronal_Fiji, 0.0)

    transform_rotation.SetParameters((eulerAngle_xyz_deg[0]/180.0*np.pi, eulerAngle_xyz_deg[1]/180.0*np.pi, eulerAngle_xyz_deg[2]/180.0*np.pi, offsets_xyz[0], offsets_xyz[1], offsets_xyz[2]))

    data_bigger = np.zeros([data.shape[0]+pad*2, data.shape[1]+pad*2, data.shape[2]+pad*2])
    data_bigger[pad:-pad, pad:-pad, pad:-pad] = data
    data_out = applyTransformToVolume(data_bigger, data_bigger, transform_rotation)

    return(data_out[pad:-pad, pad:-pad, pad:-pad])

def find_top_bottom_slices_ind(this_scan, mask_solid, ind_all_top, ind_all_bottom):

    this_scan.ind_all_margin = 100
    this_scan.ind_margin = 30
    this_scan.relative_porosity_threshold = 0.15

    mask_solid_slice_sum = np.sum(np.sum(mask_solid, axis = 2), axis = 1)
    mean_area_ind_all = np.mean(mask_solid_slice_sum[ind_all_top+this_scan.ind_all_margin:ind_all_bottom-this_scan.ind_all_margin])

    this_scan.relative_porosity = np.abs(mean_area_ind_all-mask_solid_slice_sum)/mean_area_ind_all

    this_scan.ind_top = np.min(np.where(this_scan.relative_porosity<this_scan.relative_porosity_threshold)) + this_scan.ind_margin
    this_scan.ind_bottom = np.max(np.where(this_scan.relative_porosity<this_scan.relative_porosity_threshold)) - this_scan.ind_margin

    this_scan.height_ROI = this_scan.ind_bottom-this_scan.ind_top + 1

    print('\t\tDetected ROI:\tind_top: '+str(this_scan.ind_top)+', ind_bottom: '+str(this_scan.ind_bottom)+', height: '+str(this_scan.height_ROI))
    if (this_scan.height_ROI < 200):
        print('\t\t\t\t/!\ WARNING: small height of ROI detected: check the cropping of mask solid')

    return(this_scan)

def mask_crop_ROI(mask, ind_top, ind_bottom):
    mask[:ind_top] = False
    mask[ind_bottom:] = False
    return(mask)

def close_mask_solid(this_scan, mask_solid, mask_needle, force_redo = False):

    if force_redo or not 'b_movingRegisteredToStatic_maskSolidClosed' in os.listdir(this_scan.full_path):

        # Option #1: Morphological closing to close the pores
        # chunk_size = (500, 500, 500)
        # filter_half_width = 12
        # structure = custom_3d_kernel_sphere(filter_half_width)
        # mask_solid_dilated = apply_3d_image_processing_on_subvolumes(mask_solid, fast_pytorch_mask_dilation, chunk_size_max = chunk_size, overlap = filter_half_width, radius = filter_half_width)
        # mask_solid_closed = ~apply_3d_image_processing_on_subvolumes(~mask_solid_dilated, fast_pytorch_mask_dilation, chunk_size_max = chunk_size, overlap = filter_half_width, radius = filter_half_width)

        # Option #2: Fill holes algorithm run slice by slice
        n_threads = 16
        mask_solid_closed = np.zeros_like(mask_solid)
        with multiprocessing.Pool(n_threads) as p:
            mask_solid_closed = np.array(p.map(scipy.ndimage.morphology.binary_fill_holes, mask_solid))

        # 1 core version of option #2
        # for ind, slice in enumerate(mask_solid):
        #     mask_solid_closed[ind] = scipy.ndimage.morphology.binary_fill_holes(slice)
        #     print(ind)

        mask_solid_closed[mask_needle] = False
        File(this_scan.full_path+'/b_movingRegisteredToStatic_maskSolidClosed').saveTiffStack(mask_solid_closed, type = 'bool')
    else:
        print('\tRe-loading mask solid closed')
        mask_solid_closed = File(this_scan.full_path+'/b_movingRegisteredToStatic_maskSolidClosed').readAll()

    return(mask_solid_closed.astype('bool'))

In [14]:
# Rotation
rot_coronal_Fiji = -1.0 # [Deg]
rot_sagittal_Fiji = -5.0 # [Deg]
rot_z_Fiji = 0.0 # [Deg] To get this angle, run this cell with 0.0, and adjust after sagittal /coronal rotation

# Top and bottom indices along z that contain sample for all scans (after rotation /!\)
ind_all_top = 130
ind_all_bottom = 465

In [ ]:
# Re-load scan data
with open(path_save+case+'/post_pro_data.pickle', 'rb') as handle:
    list_scans = pickle.load(handle)

nb_scans = len(list_scans)
force_redo = True

for ind_scan,this_scan in enumerate(list_scans[2:]):
    tic_scan = time.time()

    print('Running scan #'+str(ind_scan+1)+'/'+str(nb_scans)+' - '+this_scan.folder_name)

    # Loading data
    tic = time.time()
    data = File(path_save+case+'/'+this_scan.folder_name+'/b_movingRegisteredToStatic/').readAll()
    print('\tLoading data took: '+str(time.time()-tic))

    # Computing mask solid and mask needle
    mask_solid, mask_needle = compute_mask_solid(this_scan, data, rot_coronal_Fiji, rot_sagittal_Fiji, rot_z_Fiji, force_redo = force_redo)

    # Crop mask solid
    this_scan = find_top_bottom_slices_ind(this_scan, mask_solid, ind_all_top, ind_all_bottom)
    mask_solid = mask_crop_ROI(mask_solid, this_scan.ind_top, this_scan.ind_bottom)

    # Close mask_solid and compute porosity
    print('\tClosing mask solid')
    mask_solid_closed = close_mask_solid(this_scan, mask_solid, mask_needle, force_redo = force_redo)
    this_scan.porosity = 1 - np.sum(mask_solid.flatten()) / np.sum(mask_solid_closed.flatten())

    # Analyze pore structure
    mask_pores = mask_solid_closed * (~mask_solid)
    tic = time.time()
    lab = measure.label(mask_pores)
    properties = np.array(measure.regionprops(lab))
    print('\tParticle identification and analysis took: '+str(time.time()-tic))

    # Cleaning up pores
    properties = compute_height(properties)
    areas = np.array([prop.area for prop in properties])
    height = np.array([prop.height for prop in properties])
    this_scan.ind_properties_cleaned = np.where((areas>500) * (areas < 300000) * (height > 100))[0]
    properties_cleaned = properties[this_scan.ind_properties_cleaned]

    # Compute equivalent cylinder diameters
    properties_cleaned = compute_equivalent_cylinder_diameters(properties_cleaned)
    this_scan.nb_pores = len(properties_cleaned)
    # this_scan.pore_properties = properties_cleaned # Don't save this, otherwise pickle too heavy (image data stored)

    # Build and analyze skeleton for tortuosity
    tic = time.time()
    skeleton = compute_skeleton(this_scan, mask_pores, force_redo = force_redo)
    skeleton_lab = measure.label(skeleton)
    skeleton_properties = measure.regionprops(skeleton_lab)
    print('\tSkeletonize and analysis took: '+str(time.time()-tic))

    # Extract which pores to keep from skeleton
    skeleton_properties = compute_height(skeleton_properties)
    skeleton_areas = np.array([prop.area for prop in skeleton_properties])
    skeleton_height = np.array([prop.height for prop in skeleton_properties])
    this_scan.skeleton_ind_properties_cleaned = np.where((skeleton_areas>50) * (skeleton_areas < 1000) * (skeleton_height > 100))[0]

    # Compute tortuosity for each pore
    skeleton_properties_cleaned = compute_tortuosity(np.array(skeleton_properties)[this_scan.skeleton_ind_properties_cleaned])

    # Save relevant pore & skeleton properties
    this_scan.areas = [prop.area for prop in properties_cleaned]
    this_scan.equivalent_cylinder_diameters = [prop.equivalent_cylinder_diameter for prop in properties_cleaned]
    this_scan.heights = [prop.height for prop in properties_cleaned]
    this_scan.skeleton_pore_path_length = [prop.length_skeleton for prop in skeleton_properties_cleaned]
    this_scan.tortuosity = [prop.tortuosity for prop in skeleton_properties_cleaned if not np.isnan(prop.tortuosity) and prop.tortuosity > 1]

    # Update save of scan data
    print('\tUpdating save pickle')
    with open(path_save+case+'/post_pro_data.pickle', 'wb') as handle:
        pickle.dump(list_scans, handle)

    print('\tThis scan took: '+str(time.time()-tic_scan))
print('Done')


Running scan #1/38 - 03
	Loading data took: 167.43712902069092
	Rotating data took: 7.088893890380859
	3D smoothing for thresholding of mask_solid and mask_needle:
	 Processing 18 chunks of size [349 426 426] .................. Done
	 Processing 18 chunks of size [349 426 426] .................. Done
	Saving mask_solid and mask_needle:
		Detected ROI:	ind_top: 115, ind_bottom: 571, height: 457
	Closing mask solid
	Particle identification and analysis took: 13.917076587677002
	Skeletonize and analysis took: 100.96350407600403
	Updating save pickle
	This scan took: 427.74302768707275
Running scan #2/38 - 04
	Loading data took: 34.31519842147827
	Rotating data took: 8.914458990097046
	3D smoothing for thresholding of mask_solid and mask_needle:
	 Processing 18 chunks of size [349 426 426] .................. Done
	 Processing 18 chunks of size [349 426 426] .................. Done
	Saving mask_solid and mask_needle:
		Detected ROI:	ind_top: 115, ind_bottom: 572, height: 458
	Closing mask s

In [9]:
with open(path_save+case+'/post_pro_data.pickle', 'rb') as handle:
    list_scans = pickle.load(handle)

In [10]:
ind_scan = 0

print(list_scans[ind_scan].ind)
print(list_scans[ind_scan].porosity)
print(list_scans[ind_scan].nb_pores)


0
0.15139989227594763
735


In [31]:
ind_slice_X = 600-1 # Reslice from Top in Fiji
ind_slice_Y = 620-1 # Reslice from Left in Fiji
nb_scans = len(list_scans)

for ind_scan,this_scan in enumerate(list_scans):

    print('Loading scan #'+str(ind_scan+1)+'/'+str(nb_scans)+' - '+this_scan.folder_name)

    # Loading data
    tic = time.time()
    data = File(path_save+case+'/'+this_scan.folder_name+'/b_movingRegisteredToStatic/').readAll()
    print('\tLoading data took: '+str(time.time()-tic))

    File(path_save+case+'/probe_sliceX_'+str(ind_slice_X).zfill(4)).saveTiff(data[:,ind_slice_X,:], ind = this_scan.ind)
    File(path_save+case+'/probe_sliceY_'+str(ind_slice_Y).zfill(4)).saveTiff(data[:,:,ind_slice_Y], ind = this_scan.ind)

Loading scan #0/38 - 01
	Loading data took: 42.07638478279114
Loading scan #1/38 - 02
	Loading data took: 40.69148349761963
Loading scan #2/38 - 03
	Loading data took: 41.73680543899536
Loading scan #3/38 - 04
	Loading data took: 62.18978476524353
Loading scan #4/38 - 05
	Loading data took: 65.98088431358337
Loading scan #5/38 - 06
	Loading data took: 74.47796487808228
Loading scan #6/38 - 07
	Loading data took: 62.119035959243774
Loading scan #7/38 - 08a
	Loading data took: 65.75326943397522
Loading scan #8/38 - 08b
	Loading data took: 77.34974384307861
Loading scan #9/38 - 09a
	Loading data took: 60.53413009643555
Loading scan #10/38 - 09b
	Loading data took: 78.9089925289154
Loading scan #11/38 - 10a
	Loading data took: 79.08492684364319
Loading scan #12/38 - 10b
	Loading data took: 66.12908959388733
Loading scan #13/38 - 11a
	Loading data took: 49.13032126426697
Loading scan #14/38 - 11b
	Loading data took: 74.07125520706177
Loading scan #15/38 - 12a
	Loading data took: 76.23707222

In [32]:
print(time.time())

1658775671.531668
